In [1]:
infile = "../Audio/NewJeans-_뉴진스_-ETA-Official-MV-_Performance-ver._.wav"
infile = "../Audio/Michael Jackson - Beat It (Official Audio).wav"
infile = "../Audio/muzykaMJ.wav"
outfile = "../Audio/pedalboardout.wav"

In [2]:
from pedalboard import Pedalboard, Chorus, Reverb, Gain
from pedalboard.io import AudioFile
import numpy as np

# Make a Pedalboard object, containing multiple audio plugins:
room_size = 1.0
damping = 1.0
wet_level = 0.83
dry_level = 1.0
width = 1.0
freeze_mode = 0.0

room_size = 0.2
wet_level = 0.5
#Reverb(room_size, damping, wet_level, dry_level, width, freeze_mode)
board = Pedalboard([ Gain(gain_db=-24.0)])

# Open an audio file for reading, just like a regular file:
with AudioFile(infile) as f:
  
  # Open an audio file to write to:
  with AudioFile(outfile, 'w', f.samplerate, f.num_channels) as o:
  
    # Read one second of audio at a time, until the file is empty:
    i = 0
    nchunks = 259
    gain = np.arange(0, -24+(-24/nchunks), -24/nchunks)
    gain = np.arange(0, -24, -24/30)
    gain2 = np.flip(gain)
    gain = np.concatenate((gain, gain2))
    gain = np.tile(gain,int(np.ceil(nchunks/30)))

    room = np.arange(0, 1.0+1/nchunks, 1/nchunks)
    room = np.arange(0, 1.0, 1/30)
    room2 = np.flip(room)
    room = np.concatenate((room, room2))
    room = np.tile(room,int(np.ceil(nchunks/30)))




    wet = np.arange(0, 1.0+1/nchunks, 1/nchunks)
    wet = np.arange(0, 1.0, 1/30)
    wet2 = np.flip(wet)
    wet = np.concatenate((wet, wet2))
    wet = np.tile(wet,int(np.ceil(nchunks/30)))

    while f.tell() < f.frames:
      i = i+1
      chunk = f.read(f.samplerate)
      
      # Run the audio through our pedalboard:
      #board[0].room_size = room[i]
      #board[0].wet_level = wet[i]
      board[0].gain_db = gain[i]
      effected = board(chunk, f.samplerate, reset=False)
      
      # Write the output to our output file:
      o.write(effected)

In [3]:
from pedalboard import Pedalboard, Chorus, Reverb, Gain, LowpassFilter, HighShelfFilter, LowShelfFilter, HighpassFilter
from pedalboard.io import AudioFile
import numpy as np

# Make a Pedalboard object, containing multiple audio plugins:
room_size = 0.25
damping = 0.25
wet_level = 0.9
dry_level = 1.0
width = 1.0
freeze_mode = 0.0

# Reverb(room_size, damping, wet_level, dry_level, width, freeze_mode), 
board = Pedalboard([Reverb(room_size, damping, wet_level, dry_level, width, freeze_mode), 
                    HighShelfFilter(cutoff_frequency_hz=200, gain_db=-10), Gain(gain_db=-10.0)])
#board = Pedalboard([HighShelfFilter(cutoff_frequency_hz=200, gain_db=-10), Gain(gain_db=-10.0)])
#board = Pedalboard([Gain(gain_db=-10.0)])

# Open an audio file for reading, just like a regular file:
with AudioFile(infile) as f:
  
  # Open an audio file to write to:
  with AudioFile(outfile, 'w', f.samplerate, f.num_channels) as o:
    while f.tell() < f.frames:
      chunk = f.read(f.samplerate)
      effected = board(chunk, f.samplerate, reset=False)
      
      # Write the output to our output file:
      o.write(effected)

In [6]:
from numpy import convolve, vstack
import librosa as lr
import soundfile as sf

input = "../Audio/speech.wav"
input = "../Audio/NewJeans-_뉴진스_-ETA-Official-MV-_Performance-ver._.wav"
#input = "..\Audio\Michael Jackson - Beat It (Official Audio).wav"
output = "../Audio/mainoutput.wav"

degrees = 45

library_path = "../HRIR/diffuse/elev0/"
fr, frsr = sf.read(input)
if degrees > 99:
    input = library_path + "H0e" + str(degrees) + "a.wav"
else:
    input = library_path + "H0e0" + str(degrees) + "a.wav"
irfr, irsr = sf.read(input)
print(frsr, irsr)
if (frsr < irsr):
    fr = lr.resample(fr, orig_sr=frsr, target_sr=irsr)
else:
    irfr = lr.resample(irfr, orig_sr=irsr, target_sr=frsr)
    irsr = frsr
    #irfr2 = irfr
    #irfr2[:,0] = lr.resample(irfr[:,0], orig_sr=irsr, target_sr = frsr)
    #irfr2[:,1] = lr.resample(irfr[:,1], orig_sr=irsr, target_sr = frsr)
if (fr.shape[1] >= 2):
    s_R = convolve(fr[:,0],irfr[:,0])
    s_L = convolve(fr[:,1],irfr[:,1])
else:
    s_R = convolve(fr,irfr[:,0])
    s_L = convolve(fr,irfr[:,1])
mix = vstack([s_R, s_L])
print("Saving...")
sf.write(output, mix.transpose(), irsr)

44100 44100
Saving...


In [8]:
input

'../HRIR/diffuse/elev0/H0e045a.wav'